In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import os
import glob

In [ ]:
direct = '/home/erick/gw170817_fluid_profile/matter_1.0_ye'
nu_lib_table_dir = '/mnt/scratch/erick/jobs_emu/tables/NuLib_SFHo.h5'

In [ ]:
# Font settings
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'serif'
mpl.rc('text', usetex=False)

# Tick settings
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.major.pad'] = 8
mpl.rcParams['xtick.minor.size'] = 4

mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 4
mpl.rcParams['ytick.minor.width'] = 2

# Axis linewidth
mpl.rcParams['axes.linewidth'] = 2

# Tick direction and enabling ticks on all sides
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

# Function to apply custom tick locators and other settings to an Axes object
def apply_custom_settings(ax, log_scale_y=False):

    if log_scale_y:
        # Use LogLocator for the y-axis if it's in log scale
        ax.set_yscale('log')
        ax.yaxis.set_major_locator(LogLocator(base=10.0))
        ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs='auto', numticks=100))
    else:
        # Use AutoLocator for regular scales
        ax.yaxis.set_major_locator(AutoLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    # Apply the AutoLocator for the x-axis
    ax.xaxis.set_major_locator(AutoLocator())
    ax.xaxis.set_minor_locator(AutoMinorLocator())

# This function creates a pcolormesh plot with contour lines and saves it as a PNG file.
def plot_pcolormesh_with_contour(x, y, z, min_cb, max_cb, x_label, y_label, title, cbar_label, colormap, filename, do_show=True, do_save=False):

    fig, ax = plt.subplots(figsize=(12, 8))

    # Normalize Z data
    min_cb_scaled = min_cb
    max_cb_scaled = max_cb

    # Plot pcolormesh
    # Set non-data points (NaN) to black in the colormap
    cmap = plt.get_cmap(colormap).copy()
    cmap.set_bad(color='black')

    # Plot pcolormesh with masked array to handle NaNs
    c = ax.pcolormesh(x, y, z, shading='auto', cmap=cmap, vmin=min_cb_scaled, vmax=max_cb_scaled)

    # Add contour lines
    # contour = ax.contour(x, y, z, colors='black', linewidths=1.5, levels=3)
    # ax.clabel(contour, inline=True, fontsize=15, fmt='%1.1f')
    
    # Plot settings
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    # ax.set_title(title+'\nmin: {:.2e}\nmax: {:.2e}'.format(np.nanmin(z), np.nanmax(z)))
    ax.set_title(title)

    # Add color bar
    cbar = fig.colorbar(c, ax=ax, label=cbar_label)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    apply_custom_settings(ax, False)

    # Ensure equal aspect ratio
    ax.set_aspect('equal', 'box')

    # Save figure
    if do_save:
        fig.savefig(direct+'/plots/'+filename, format='png', bbox_inches='tight')

    # Display figure
    if do_show:
        plt.show()
        # display(fig)
    
    # Close figure
    plt.close(fig)

In [ ]:
rho_ye_T_h5py = h5py.File(direct+'/rho_Ye_T.hdf5', 'r')

# number of cells
Nx = np.array(rho_ye_T_h5py['/ncellsx'])
Ny = np.array(rho_ye_T_h5py['/ncellsy'])
Nz = np.array(rho_ye_T_h5py['/ncellsz'])
print(f'Nx = {Nx}, Ny = {Ny}, Nz = {Nz}')

# cell size
dx = ( np.array(rho_ye_T_h5py['/xmax_cm']) - np.array(rho_ye_T_h5py['/xmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsx']) # cm
dy = ( np.array(rho_ye_T_h5py['/ymax_cm']) - np.array(rho_ye_T_h5py['/ymin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsy']) # cm
dz = ( np.array(rho_ye_T_h5py['/zmax_cm']) - np.array(rho_ye_T_h5py['/zmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsz']) # cm
print(f'dx = {dx} cm, dy = {dy} cm, dz = {dz} cm')

# cell centers
xc = np.linspace(dx / 2, dx * (Nx - 0.5), Nx) # cm
yc = np.linspace(dy / 2, dy * (Ny - 0.5), Ny) # cm
zc = np.linspace(dz / 2, dz * (Nz - 0.5), Nz) # cm

# cell centers mesh
Xc, Yc, Zc = np.meshgrid(xc, yc, zc, indexing='ij') # cm

bh_radius = 5.43e+05 # cm
bh_center_x = 48.0e+5 # cm
bh_center_y = 48.0e+5 # cm
bh_center_z = 16.0e+5 # cm

distance_from_bh = np.sqrt( (Xc - bh_center_x)**2 + (Yc - bh_center_y)**2 + (Zc - bh_center_z)**2 )
mask_bh = (distance_from_bh < bh_radius)
# mask_x = (Xc < dx) | (Xc > (Nx - 1) * dx)
# mask_y = (Yc < dy) | (Yc > (Ny - 1) * dy)
# mask_z = (Zc < dz) | (Zc > (Nz - 1) * dz)
# mask = mask_bh | mask_x | mask_y | mask_z
mask = mask_bh

In [ ]:
x_slice_indx = 48 # cell number
y_slice_indx = 48 # cell number
z_slice_indx = 16 # cell number

z_slice_value = Zc[0,0,z_slice_indx] # cm
y_slice_value = Yc[0,y_slice_indx,0] # cm

# Data x and y for upper panel
xup = Xc[:, :, z_slice_indx] / 1e5 # km
yup = Yc[:, :, z_slice_indx] / 1e5 # km

# Data x and y for lower panel
xlow = Xc[:, y_slice_indx, :] / 1e5 # km
ylow = Zc[:, y_slice_indx, :] / 1e5 # km

In [ ]:
print(f"T_Mev  = {np.array(rho_ye_T_h5py['/T_Mev'])[32,48,15]}")
print(f"rho_g|ccm  = {np.array(rho_ye_T_h5py['/rho_g|ccm'])[32,48,15]}")
print(f"Ye  = {np.array(rho_ye_T_h5py['/Ye'])[32,48,15]}")

# TEMPERATURE

In [ ]:
temp_slice_up = np.array(rho_ye_T_h5py['/T_Mev'])    [:,:,z_slice_indx]
temp_slice_low = np.array(rho_ye_T_h5py['/T_Mev'])    [:,y_slice_indx,:]

max_color_bar = np.max([np.max(temp_slice_low), np.max(temp_slice_up)])
min_color_bar = np.min([np.min(temp_slice_low), np.min(temp_slice_up)])

x = xup.copy()
y = yup.copy()
z = temp_slice_up.copy()
z[mask[:,:,z_slice_indx]] = np.nan

plot_pcolormesh_with_contour(x=x, 
                            y=y, 
                            z=z, 
                            min_cb=min_color_bar, 
                            max_cb=max_color_bar, 
                            x_label=r'$x \, (\mathrm{km})$', 
                            y_label=r'$y \, (\mathrm{km})$', 
                            title=fr'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                            cbar_label=fr'$ T\,(\mathrm{{MeV}})$', 
                            colormap='viridis',
                            filename=f'temperature_map_yx.png',
                            do_show=True,
                            do_save=True)

x = xlow.copy()
y = ylow.copy()
z = temp_slice_low.copy()
z[mask[:,y_slice_indx,:]] = np.nan

plot_pcolormesh_with_contour(x=x, 
                            y=y, 
                            z=z, 
                            min_cb=min_color_bar, 
                            max_cb=max_color_bar, 
                            x_label=r'$x \, (\mathrm{km})$', 
                            y_label=r'$z \, (\mathrm{km})$', 
                            title=rf'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                            cbar_label=fr'$ T\,(\mathrm{{MeV}})$', 
                            colormap='viridis',
                            filename=f'temperature_map_zx.png',
                            do_show=True,
                            do_save=True)

# ELECTRON FRACTION

In [ ]:
ye_slice_up   = np.array(rho_ye_T_h5py['/Ye'])       [:,:,z_slice_indx]
ye_slice_low   = np.array(rho_ye_T_h5py['/Ye'])       [:,y_slice_indx,:]

max_color_bar = np.max([np.max(ye_slice_low), np.max(ye_slice_up)])
min_color_bar = np.min([np.min(ye_slice_low), np.min(ye_slice_up)])

x = xup.copy()
y = yup.copy()
z = ye_slice_up.copy()
z[mask[:,:,z_slice_indx]] = np.nan

plot_pcolormesh_with_contour(x=x, 
                            y=y, 
                            z=z, 
                            min_cb=min_color_bar, 
                            max_cb=max_color_bar, 
                            x_label=r'$x \, (\mathrm{km})$', 
                            y_label=r'$y \, (\mathrm{km})$', 
                            title=fr'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                            cbar_label=fr'$Ye$', 
                            colormap='viridis',
                            filename=f'ye_map_yx.png',
                            do_show=True,
                            do_save=True)
x = xlow.copy()
y = ylow.copy()
z = ye_slice_low.copy()
z[mask[:,y_slice_indx,:]] = np.nan

plot_pcolormesh_with_contour(x=x, 
                            y=y, 
                            z=z, 
                            min_cb=min_color_bar, 
                            max_cb=max_color_bar, 
                            x_label=r'$x \, (\mathrm{km})$', 
                            y_label=r'$z \, (\mathrm{km})$', 
                            title=rf'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                            cbar_label=fr'$Ye$', 
                            colormap='viridis',
                            filename=f'ye_map_zx.png',
                            do_show=True,
                            do_save=True)

# DENSITY

In [ ]:
rho_slice_up  = np.array(rho_ye_T_h5py['/rho_g|ccm'])[:,:,z_slice_indx]
rho_slice_low  = np.array(rho_ye_T_h5py['/rho_g|ccm'])[:,y_slice_indx,:]

# max_color_bar = np.max([np.max(rho_slice_low), np.max(rho_slice_up)]) / 1e11
# min_color_bar = np.min([np.min(rho_slice_low), np.min(rho_slice_up)]) / 1e11

max_color_bar = np.max([np.max(np.log10(rho_slice_low)), np.max(np.log10(rho_slice_up))])
min_color_bar = np.min([np.min(np.log10(rho_slice_low)), np.min(np.log10(rho_slice_up))])

x = xup.copy()
y = yup.copy()
z = rho_slice_up.copy() / 1e11
z = np.log10( rho_slice_up.copy() )
z[mask[:,:,z_slice_indx]] = np.nan

plot_pcolormesh_with_contour(x=x, 
                            y=y, 
                            z=z, 
                            min_cb=min_color_bar, 
                            max_cb=max_color_bar, 
                            # min_cb=None, 
                            # max_cb=None,
                            x_label=r'$x \, (\mathrm{km})$',
                            y_label=r'$y \, (\mathrm{km})$', 
                            title=fr'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                            cbar_label=fr'$\log(\rho \, [\mathrm{{g/cm}}^{3}])$', 
                            colormap='viridis',
                            filename=f'rho_map_yx.png',
                            do_show=True,
                            do_save=True)
x = xlow.copy()
y = ylow.copy()
z = np.log10( rho_slice_low.copy() )
z[mask[:,y_slice_indx,:]] = np.nan

plot_pcolormesh_with_contour(x=x, 
                            y=y, 
                            z=z, 
                            min_cb=min_color_bar, 
                            max_cb=max_color_bar,
                            # min_cb=None, 
                            # max_cb=None,
                            x_label=r'$x \, (\mathrm{km})$', 
                            y_label=r'$z \, (\mathrm{km})$', 
                            title=rf'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                            cbar_label=fr'$\log(\rho \, [\mathrm{{g/cm}}^{3}])$', 
                            colormap='viridis',
                            filename=f'rho_map_zx.png',
                            do_show=True,
                            do_save=True)

# PLOT ENERGY DEPENDENT QUANTITIES

In [ ]:
files_multiple_quantities = [
    'n_nu_e_eq_ccm',
    'n_nubar_e_eq_ccm',
    'n_nu_x_eq_ccm',
    'nu_e_absorption_opacity_cm',
    'nubar_e_absorption_opacity_cm',
    'nu_x_absorption_opacity_cm',
    'nu_e_scattering_opacity_cm',
    'nubar_e_scattering_opacity_cm',
    'nu_x_scattering_opacity_cm',
    'nu_e_absorption_optical_depth_1km',
    'nubar_e_absorption_optical_depth_1km',
    'nu_x_absorption_optical_depth_1km',
    'nu_e_scattering_optical_depth_1km',
    'nubar_e_scattering_optical_depth_1km',
    'nu_x_scattering_optical_depth_1km'
    ]

files_multiple_quantities_latex = [
    r'$n_{\nu_e}^{\mathrm{eq}}$ $\mathrm{(cm^{-3})}$',                     # n_nu_e_eq_ccm
    r'$n_{\bar{\nu}_e}^{\mathrm{eq}}$ $\mathrm{(cm^{-3})}$',               # n_nubar_e_eq_ccm
    r'$n_{\nu_x}^{\mathrm{eq}}$ $\mathrm{(cm^{-3})}$',                     # n_nu_x_eq_ccm
    r'$\kappa_{\nu_e}^{\mathrm{abs}}$ $\mathrm{(cm^{-1})}$',           # nu_e_absorption_opacity_cm
    r'$\kappa_{\bar{\nu}_e}^{\mathrm{abs}}$ $\mathrm{(cm^{-1})}$',     # nubar_e_absorption_opacity_cm
    r'$\kappa_{\nu_x}^{\mathrm{abs}}$ $\mathrm{(cm^{-1})}$',           # nu_x_absorption_opacity_cm
    r'$\kappa_{\nu_e}^{\mathrm{scat}}$ $\mathrm{(cm^{-1})}$',          # nu_e_scattering_opacity_cm
    r'$\kappa_{\bar{\nu}_e}^{\mathrm{scat}}$ $\mathrm{(cm^{-1})}$',    # nubar_e_scattering_opacity_cm
    r'$\kappa_{\nu_x}^{\mathrm{scat}}$ $\mathrm{(cm^{-1})}$',          # nu_x_scattering_opacity_cm
    r'$\tau_{\nu_e}^{\mathrm{abs}}$ $\mathrm{(s=1\,km)}$',               # nu_e_absorption_optical_depth_1km
    r'$\tau_{\bar{\nu}_e}^{\mathrm{abs}}$ $\mathrm{(s=1\,km)}$',        # nubar_e_absorption_optical_depth_1km
    r'$\tau_{\nu_x}^{\mathrm{abs}}$ $\mathrm{(s=1\,km)}$',              # nu_x_absorption_optical_depth_1km
    r'$\tau_{\nu_e}^{\mathrm{scat}}$ $\mathrm{(s=1\,km)}$',             # nu_e_scattering_optical_depth_1km
    r'$\tau_{\bar{\nu}_e}^{\mathrm{scat}}$ $\mathrm{(s=1\,km)}$',       # nubar_e_scattering_optical_depth_1km
    r'$\tau_{\nu_x}^{\mathrm{scat}}$ $\mathrm{(s=1\,km)}$'              # nu_x_scattering_optical_depth_1km
 ]

nulib_h5py = h5py.File(nu_lib_table_dir, 'r')
E_bins_MeV = np.array(nulib_h5py['/neutrino_energies']) # MeV
nulib_h5py.close()

for i, file_name in enumerate(files_multiple_quantities):
# for i, file_name in enumerate(files_multiple_quantities[0:1]):

    print(f'file_name = {file_name}')

    filename = file_name
    cbarlabel = files_multiple_quantities_latex[i]
    file_h5py = h5py.File(direct+'/'+filename+'.h5', 'r')
    quantity = np.array(file_h5py['/'+filename])
    quantity[:,mask] = np.nan

    for i in range(len(quantity)):
    # for i in [0]:

        plot_pcolormesh_with_contour(x=xup, 
                                    y=yup, 
                                    z=quantity[i,:,:,z_slice_indx], 
                                    min_cb=None, 
                                    max_cb=None, 
                                    x_label=r'$x \, (\mathrm{km})$',
                                    y_label=r'$y \, (\mathrm{km})$', 
                                    title=f'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$ \n $E = {E_bins_MeV[i]:.2f}$ MeV',
                                    cbar_label=cbarlabel, 
                                    colormap='viridis',
                                    filename=filename+'_energybin'+str(i)+'_yx.png',
                                    do_show=False,
                                    do_save=False)
        
        plot_pcolormesh_with_contour(x=xlow, 
                                    y=ylow, 
                                    z=quantity[i,:,y_slice_indx,:], 
                                    min_cb=None, 
                                    max_cb=None, 
                                    x_label=r'$x \, (\mathrm{km})$', 
                                    y_label=r'$z \, (\mathrm{km})$', 
                                    title=f'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$ \n $E = {E_bins_MeV[i]:.2f}$ MeV',
                                    cbar_label=cbarlabel, 
                                    colormap='viridis',
                                    filename=filename+'_energybin'+str(i)+'_zx.png',
                                    do_show=False,
                                    do_save=False)
    file_h5py.close()

# PLOT ENERGY INDEPENDENT QUANTITIES

In [ ]:
file_names_chem_pot = [
    'mu_nu_e_MeV', # nu_e chemical potential
    'mu_nubar_e_MeV' # nu_e chemical potential
    ]

file_names_chem_pot_latex = [
    r'$\mu_{\nu_e} \, (\mathrm{MeV})$', # nu_e chemical potential
    r'$\mu_{\bar{\nu}_e} \, (\mathrm{MeV})$' # nu_e chemical potential
    ]

for i, file_name in enumerate(file_names_chem_pot):
# for i, file_name in enumerate(file_names_chem_pot[0:1]):

    print(f'file_name = {file_name}')

    filename = file_name
    cbarlabel = file_names_chem_pot_latex[i]
    file_h5py = h5py.File(direct+'/'+filename+'.h5', 'r')
    quantity = np.array(file_h5py['/'+filename])
    quantity[mask] = np.nan

    print(f"maxcbar = {np.nanmax(quantity)}")
    print(f"mincbar = {np.nanmin(quantity)}")

    maxcbar = 4.850901928160308
    mincbar = -5.625765869343175

    min_abs_cbar = min(abs(maxcbar), abs(mincbar))

    plot_pcolormesh_with_contour(x=xup, 
                                y=yup, 
                                z=quantity[:,:,z_slice_indx], 
                                min_cb=-min_abs_cbar, 
                                max_cb=min_abs_cbar, 
                                x_label=r'$x \, (\mathrm{km})$',
                                y_label=r'$y \, (\mathrm{km})$', 
                                title=f'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                                cbar_label=cbarlabel, 
                                colormap='seismic',
                                filename=filename+'_yx.png',
                                do_show=True, 
                                do_save=True)

    plot_pcolormesh_with_contour(x=xlow, 
                                y=ylow, 
                                z=quantity[:,y_slice_indx,:], 
                                min_cb=-min_abs_cbar, 
                                max_cb=min_abs_cbar, 
                                x_label=r'$x \, (\mathrm{km})$', 
                                y_label=r'$z \, (\mathrm{km})$', 
                                title=f'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                                cbar_label=cbarlabel, 
                                colormap='seismic',
                                filename=filename+'_zx.png',
                                do_show=True, 
                                do_save=True)    
    file_h5py.close()

# COMPUTE EMISSION RATES AND LUMINOSITIES PER ENERGY BIN

In [ ]:
one_cubic_km = 1e5**3 # cm^3
c = 2.99792458e+10 # cm/s
eV = 1.60218e-12  # erg

output_file_names = ['N_dot_nu_e_eq','N_dot_nubar_e_eq','N_dot_nu_x_eq']
cbarlabels = [r'$\dot{N}_{\nu_e}^{\mathrm{em}}$ ($s^{-1}$)',r'$\dot{N}_{\bar{\nu}_e}^{\mathrm{em}}$ ($s^{-1}$)',r'$\dot{N}_{\nu_x}^{\mathrm{em}}$ ($s^{-1}$)']
n_ccm_file_names = ['n_nu_e_eq_ccm','n_nubar_e_eq_ccm','n_nu_x_eq_ccm']
opacity_cm_file_names = ['nu_e_absorption_opacity_cm','nubar_e_absorption_opacity_cm','nu_x_absorption_opacity_cm']
freestreamingbins = [(0,6),(0,11),(0,None)]

for j in range(3):
# for j in [0]:

    n_ccm_h5py = h5py.File(direct+'/'+n_ccm_file_names[j]+'.h5', 'r')
    n_ccm = np.array(n_ccm_h5py['/'+n_ccm_file_names[j]])

    opacity_cm_h5py = h5py.File(direct+'/'+opacity_cm_file_names[j]+'.h5', 'r')
    opacity_cm = np.array(opacity_cm_h5py['/'+opacity_cm_file_names[j]])

    N_dot = n_ccm * opacity_cm * one_cubic_km * c
    L_egs_s = N_dot * E_bins_MeV[:, np.newaxis, np.newaxis, np.newaxis] * 1.0e6 * eV # erg/s

    N_dot[:,mask] = np.nan
    L_egs_s[:,mask] = np.nan

    print(f'{output_file_names[j]} luminosity = {                np.sum(L_egs_s[freestreamingbins[j][0]:freestreamingbins[j][1],~mask]):.2e} erg/s')
    print(f'{output_file_names[j]} free_streeming_energy_bins = {np.sum(N_dot  [freestreamingbins[j][0]:freestreamingbins[j][1],~mask]):.2e} 1/s')

    filename = output_file_names[j]
    cbarlabel = cbarlabels[j]

    # for i in range(len(N_dot)):
    for i in [0]:

        print(f'E = {E_bins_MeV[i]:.2f} MeV. Total {output_file_names[j]} = {np.sum(N_dot[i,~mask]):.2e} 1/s')

        plot_pcolormesh_with_contour(x=xup, 
                                    y=yup, 
                                    z=N_dot[i,:,:,z_slice_indx],
                                    min_cb=None,
                                    max_cb=None,
                                    x_label=r'$x \, (\mathrm{km})$',
                                    y_label=r'$y \, (\mathrm{km})$',
                                    title=f'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$ \n $E = {E_bins_MeV[i]:.2f}$ MeV',
                                    cbar_label=cbarlabel,
                                    colormap='viridis',
                                    filename=filename+'_energybin'+str(i)+'_yx.png',
                                    do_show=False, 
                                    do_save=False)
        
        plot_pcolormesh_with_contour(x=xlow, 
                                    y=ylow, 
                                    z=N_dot[i,:,y_slice_indx,:], 
                                    min_cb=None, 
                                    max_cb=None, 
                                    x_label=r'$x \, (\mathrm{km})$', 
                                    y_label=r'$z \, (\mathrm{km})$', 
                                    title=f'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$ \n $E = {E_bins_MeV[i]:.2f}$ MeV',
                                    cbar_label=cbarlabel, 
                                    colormap='viridis',
                                    filename=filename+'_energybin'+str(i)+'_zx.png',
                                    do_show=True, 
                                    do_save=False)
                
    n_ccm_h5py.close()
    opacity_cm_h5py.close()    

# COMPUTE TOTAL EMISSION RATES

In [ ]:
one_cubic_km = 1e5**3 # cm^3
c = 2.99792458e+10 # cm/s
eV = 1.60218e-12  # erg

output_file_names = ['N_dot_nu_e_eq','N_dot_nubar_e_eq','N_dot_nu_x_eq']
cbarlabels = [r'$\log(\dot{N}_{\nu_e}^{\mathrm{em}}\,[s^{-1}])$',r'$\log(\dot{N}_{\bar{\nu}_e}^{\mathrm{em}}\,[s^{-1}])$',r'$\log(\dot{N}_{\nu_x}^{\mathrm{em}}\,[s^{-1}])$']
n_ccm_file_names = ['n_nu_e_eq_ccm','n_nubar_e_eq_ccm','n_nu_x_eq_ccm']
opacity_cm_file_names = ['nu_e_absorption_opacity_cm','nubar_e_absorption_opacity_cm','nu_x_absorption_opacity_cm']
freestreamingbins = [(0,6),(0,11),(0,None)]

for j in range(3):
# for j in [0]:

    n_ccm_h5py = h5py.File(direct+'/'+n_ccm_file_names[j]+'.h5', 'r')
    n_ccm = np.array(n_ccm_h5py['/'+n_ccm_file_names[j]])

    opacity_cm_h5py = h5py.File(direct+'/'+opacity_cm_file_names[j]+'.h5', 'r')
    opacity_cm = np.array(opacity_cm_h5py['/'+opacity_cm_file_names[j]])

    N_dot = n_ccm * opacity_cm * one_cubic_km * c

    L_egs_s = N_dot * E_bins_MeV[:, np.newaxis, np.newaxis, np.newaxis] * 1.0e6 * eV # erg/s

    N_dot[:,mask] = np.nan
    L_egs_s[:,mask] = np.nan

    print(f'{output_file_names[j]} luminosity = {                np.sum(L_egs_s[freestreamingbins[j][0]:freestreamingbins[j][1],~mask]):.2e} erg/s')
    print(f'{output_file_names[j]} free_streeming_energy_bins = {np.sum(N_dot  [freestreamingbins[j][0]:freestreamingbins[j][1],~mask]):.2e} 1/s')

    filename = output_file_names[j]
    cbarlabel = cbarlabels[j]

    print("N_dot.shape=", N_dot.shape)

    # N_dot[2:,:,:,:] = 0.0 # remove the bin that are not used in the simulation
    # N_dot[0,:,:,:] = 0.0 # remove the bin that are not used in the simulation

    N_dot_sum_bins = np.sum(N_dot[:13, :, :, :], axis=0)  # sum over the first 13 energy bins

    print("N_dot_sum_bins.shape=", N_dot_sum_bins.shape)

    plot_pcolormesh_with_contour(x=xup, 
                                y=yup, 
                                # z=N_dot_sum_bins[:,:,z_slice_indx],
                                z=np.log10(N_dot_sum_bins[:,:,z_slice_indx]),
                                # min_cb=None, 
                                # max_cb=None,
                                min_cb=44,
                                max_cb=54,
                                x_label=r'$x \, (\mathrm{km})$',
                                y_label=r'$y \, (\mathrm{km})$',
                                title=f'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                                cbar_label=cbarlabel,
                                colormap='viridis',
                                filename=filename+'_total_yx.png',
                                do_show=True, 
                                do_save=True)
    
    plot_pcolormesh_with_contour(x=xlow, 
                                y=ylow, 
                                # z=N_dot_sum_bins[:,y_slice_indx,:], 
                                z=np.log10(N_dot_sum_bins[:,y_slice_indx,:]),
                                # min_cb=None, 
                                # max_cb=None, 
                                min_cb=44,
                                max_cb=54,
                                x_label=r'$x \, (\mathrm{km})$', 
                                y_label=r'$z \, (\mathrm{km})$', 
                                title=f'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                                cbar_label=cbarlabel, 
                                colormap='viridis',
                                filename=filename+'_total_zx.png',
                                do_show=True, 
                                do_save=True)

    n_ccm_h5py.close()
    opacity_cm_h5py.close()    

In [ ]:
one_cubic_km = 1e5**3 # cm^3
c = 2.99792458e+10 # cm/s
eV = 1.60218e-12  # erg

n_e_ccm_h5py = h5py.File(direct+'/n_nu_e_eq_ccm'+'.h5', 'r')
n_e_ccm = np.array(n_e_ccm_h5py['/n_nu_e_eq_ccm'])

nbar_e_ccm_h5py = h5py.File(direct+'/n_nubar_e_eq_ccm'+'.h5', 'r')
nbar_e_ccm = np.array(nbar_e_ccm_h5py['/n_nubar_e_eq_ccm'])

n_e_opacity_cm_h5py = h5py.File(direct+'/nu_e_absorption_opacity_cm'+'.h5', 'r')
n_e_opacity_cm = np.array(n_e_opacity_cm_h5py['/nu_e_absorption_opacity_cm'])

nbar_e_opacity_cm_h5py = h5py.File(direct+'/nubar_e_absorption_opacity_cm'+'.h5', 'r')
nbar_e_opacity_cm = np.array(nbar_e_opacity_cm_h5py['/nubar_e_absorption_opacity_cm'])

N_e_dot = n_e_ccm * n_e_opacity_cm * one_cubic_km * c # 1/s
Nbar_e_dot = nbar_e_ccm * nbar_e_opacity_cm * one_cubic_km * c # 1/s

N_e_dot[:,mask] = np.nan
Nbar_e_dot[:,mask] = np.nan

ratio_N_e_dot_Nbar_e_dot = N_e_dot / Nbar_e_dot
Nbar_e_dot[:,mask] = np.nan

cbarlabel = r'$\dot{N}_{\nu_e}^{\mathrm{em}} / \dot{N}_{\bar{\nu}_e}^{\mathrm{em}}$'
filename = 'ratio_N_e_dot_Nbat_e_dot'

for i in range(len(N_dot)):
# for i in [5]:

    plot_pcolormesh_with_contour(x=xup, 
                                y=yup, 
                                z=ratio_N_e_dot_Nbar_e_dot[i,:,:,z_slice_indx],
                                min_cb=0.5,
                                max_cb=1.5,
                                x_label=r'$x \, (\mathrm{km})$',
                                y_label=r'$y \, (\mathrm{km})$',
                                title=f'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$ \n $E = {E_bins_MeV[i]:.2f}$ MeV',
                                cbar_label=cbarlabel,
                                colormap='seismic',
                                filename=filename+'_energybin'+str(i)+'_yx.png',
                                do_show=False, 
                                do_save=False)    
    plot_pcolormesh_with_contour(x=xlow, 
                                y=ylow, 
                                z=ratio_N_e_dot_Nbar_e_dot[i,:,y_slice_indx,:], 
                                min_cb=0.5,
                                max_cb=1.5,
                                x_label=r'$x \, (\mathrm{km})$', 
                                y_label=r'$z \, (\mathrm{km})$', 
                                title=f'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$ \n $E = {E_bins_MeV[i]:.2f}$ MeV',
                                cbar_label=cbarlabel, 
                                colormap='seismic',
                                filename=filename+'_energybin'+str(i)+'_zx.png',
                                do_show=False, 
                                do_save=False)